In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling1D

# Load the dataset
file_path = "output.csv"  # Update this with your actual file path
df = pd.read_csv(file_path)

df.columns = df.columns.str.strip()
print(df.columns)

# Drop non-numeric columns
non_numeric_cols = ['Device ID', 'Warm?', 'Sync', 'Arm', 'Timestamp', 'Pose', 'Locked', 'RSSI', 'newTimeStamp']
df = df.drop(columns=non_numeric_cols, errors='ignore')

# Encode the target labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])  # Convert 'release' and 'fisk' to numbers

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(df.drop(columns=['label']))
y = tf.keras.utils.to_categorical(df['label'])  # One-hot encode labels

# Reshape for CNN (samples, time steps, features)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build CNN Model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X.shape[1], 1)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    
    # Global Average Pooling instead of Flatten to avoid shape mismatches
    GlobalAveragePooling1D(),
    
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y.shape[1], activation='softmax')  # Output layer for classification
])

# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train Model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluate Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

# Save Model
model.save("gesture_cnn_model.h5")


Index(['Unnamed: 0', 'Device ID', 'Warm?', 'Sync', 'Arm', 'Timestamp',
       'Orientation_W', 'Orientation_X', 'Orientation_Y', 'Orientation_Z',
       'Acc_X', 'Acc_Y', 'Acc_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z', 'Pose',
       'EMG_1', 'EMG_2', 'EMG_3', 'EMG_4', 'EMG_5', 'EMG_6', 'EMG_7', 'EMG_8',
       'Locked', 'RSSI', 'Roll', 'Pitch', 'Yaw', 'newTimeStamp',
       'elapsed_seconds', 'label'],
      dtype='object')
Epoch 1/20


c:\Users\haris\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


430/430 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.5808 - loss: 0.7272 - val_accuracy: 0.6152 - val_loss: 0.6585
Epoch 2/20
430/430 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6800 - loss: 0.5841 - val_accuracy: 0.7240 - val_loss: 0.5503
Epoch 3/20
430/430 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7153 - loss: 0.5485 - val_accuracy: 0.7350 - val_loss: 0.5152
Epoch 4/20
430/430 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7477 - loss: 0.5062 - val_accuracy: 0.7618 - val_loss: 0.4919
Epoch 5/20
430/430 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7675 - loss: 0.4859 - val_accuracy: 0.7702 - val_loss: 0.4784
Epoch 6/20
430/430 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7796 - loss: 0.4555 - val_accuracy: 0.7859 - val_loss: 0.4501
Epoch 7/20
430/430 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7962 - loss: 0.4289 - val_accuracy: 0.7644 - val_loss: 0.4836
Epoch 8/20
430/430 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8072 - loss: 0.4226 - val_accuracy: 0.7932 - val_

Test Accuracy: 0.8519


In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.metrics import accuracy_score

# Custom transformer for reshaping data for CNN input
class ReshapeForCNN(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Reshape to (samples, time_steps, features)
        return X.reshape(X.shape[0], X.shape[1], 1)

# Load the dataset
file_path = "output.csv"  # Update this with your actual file path
df = pd.read_csv(file_path)

# Clean column names by stripping spaces
df.columns = df.columns.str.strip()

# Drop non-numeric columns
non_numeric_cols = ['Device ID', 'Warm?', 'Sync', 'Arm', 'Timestamp', 'Pose', 'Locked', 'RSSI', 'newTimeStamp']
df = df.drop(columns=non_numeric_cols, errors='ignore')

# Encode the target labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])  # Convert 'release' and 'fisk' to numbers

# Separate features and target
X = df.drop(columns=['label'])
y = df['label']

# Normalize features
scaler = StandardScaler()

# Build the preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('features', scaler, X.columns)  # Apply scaling to all feature columns
    ]
)

# Define a function to create the CNN model
def create_cnn_model(input_shape):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(np.unique(y)), activation='softmax')  # Output layer for classification
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess the data
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Reshape the data for CNN
X_train_processed = X_train_processed.reshape(X_train_processed.shape[0], X_train_processed.shape[1], 1)
X_test_processed = X_test_processed.reshape(X_test_processed.shape[0], X_test_processed.shape[1], 1)

# Build and train the model
model = create_cnn_model(input_shape=(X_train_processed.shape[1], 1))
model.fit(X_train_processed, tf.keras.utils.to_categorical(y_train), epochs=20, batch_size=32, validation_data=(X_test_processed, tf.keras.utils.to_categorical(y_test)))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_processed, tf.keras.utils.to_categorical(y_test))
print(f"Test Accuracy: {accuracy:.4f}")

# Save Model
model.save("gesture_cnn_model.h5")


Epoch 1/20


c:\Users\haris\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


430/430 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6224 - loss: 0.8004 - val_accuracy: 0.7103 - val_loss: 0.5438
Epoch 2/20
430/430 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7347 - loss: 0.5199 - val_accuracy: 0.7903 - val_loss: 0.4596
Epoch 3/20
430/430 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7703 - loss: 0.4604 - val_accuracy: 0.8159 - val_loss: 0.4023
Epoch 4/20
430/430 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8057 - loss: 0.4159 - val_accuracy: 0.8144 - val_loss: 0.4019
Epoch 5/20
430/430 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8307 - loss: 0.3689 - val_accuracy: 0.8421 - val_loss: 0.3487
Epoch 6/20
430/430 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8425 - loss: 0.3479 - val_accuracy: 0.8371 - val_loss: 0.3534
Epoch 7/20
430/430 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8529 - loss: 0.3295 - val_accuracy: 0.8519 - val_loss: 0.3377
Epoch 8/20
430/430 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8597 - loss: 0.3078 - val_accuracy: 0.8551 - val_

Test Accuracy: 0.8828


In [29]:
pd.set_option('display.max_rows', None)  # None will display all rows
pd.set_option('display.max_columns', None)  # None will display all columns
pd.set_option('display.width', None)  # Adjust width to fit all columns
pd.set_option('display.max_colwidth', None) 

In [13]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler # Assuming you're importing pandas for data loading
import pandas as pd
from scipy import stats

# Set chunk size for reading the file incrementally
chunk_size = 200 # Read 200 rows at a time

# Initialize a counter to track the number of rows read
rows_read = 0

# Placeholder for processed data (you can choose to append or process incrementally as needed)
X_list = []

# Define non-numeric columns to drop
non_numeric_cols = ['Device ID', 'Warm?', 'Sync', 'Arm', 'Timestamp', 'Pose', 'Locked', 'RSSI', 'newTimeStamp']

# Define a function for processing each chunk
def process_chunk(chunk, scaler):
    # Clean column names
    chunk.columns = chunk.columns.str.strip()

    # Drop non-numeric columns
    chunk = chunk.drop(columns=non_numeric_cols, errors='ignore')

    # Normalize features
    X = chunk  # Use the remaining columns as features
    X_scaled = scaler.transform(X)  # Apply the scaler (assuming scaler is fit on training data)

    # Reshape data for CNN (samples, time steps, features)
    X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)  # Reshape to (samples, time_steps, features)

    return X_scaled

# Initialize the scaler (you can fit it on your training data before this step)
scaler = StandardScaler()
model = tf.keras.models.load_model("gesture_cnn_model.h5")

def get_mode(array):
    mode_result = stats.mode(array)  # This returns an object with mode values and counts
    mode = mode_result.mode[0]  # Corrected: access the mode value using .mode
    return mode

# Open the CSV and read in chunks
while True:
    for chunk in pd.read_csv('fisk_emg_filename.csv', chunksize=chunk_size):
        # If this is the first chunk, fit the scaler on the data (using the training data ideally)
        if rows_read == 0:
            # Fit the scaler on the chunk (or better, fit on training data if available)
            chunk.columns = chunk.columns.str.strip()
            scaler.fit(chunk.drop(columns=non_numeric_cols, errors='ignore'))

        # Process each chunk
        X_chunk = process_chunk(chunk, scaler)

        # Store or use the processed data incrementally
        X_list.append(X_chunk)


        predictions = model.predict(X_chunk)

        # If you need to convert predictions back to original label encoding:
        predicted_labels = np.argmax(predictions, axis=1)

        # Set NumPy to print full arrays without truncating
        np.set_printoptions(threshold=np.inf)

        # Print the full predictions array
        
        print(stats.mode(predicted_labels))


        # Print progress (optional)
        rows_read += len(chunk)

    # Convert the list to a numpy array for prediction
    X_full = np.concatenate(X_list, axis=0)

    # Now you can use X_full for model prediction
    print(f"Processed {rows_read} rows in total.")

    # Load the trained model


    # Make predictions


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_6" is incompatible with the layer: expected axis -1 of input shape to have value 512, but received input with shape (32, 384)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 21, 1), dtype=float32)
  • training=False
  • mask=None